# #Preprocessing the Data


step1: delete the column from the dataframe that have large number of NAN value or missing value(more then 70% of datapoints)


step2: In the column "beer/name", "beer/style", "user/profilename","user/gender" assign number using Labelencoder


step3: in the column "review/overall" consider as output and rest all column consider as input


In [276]:
#import all important libreries
import csv
import numpy as np 
import pandas as pd 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.preprocessing import LabelEncoder

#read the data

dff=pd.read_csv('train.csv')

df=dff.drop(['review/overall','review/text','review/timeStruct','user/birthdayRaw','user/ageInSeconds','user/birthdayUnix'], axis=1)

data = np.array(df) 
labelencoder = LabelEncoder()
data[:, 4] = labelencoder.fit_transform(data[:, 4])
data[:, 5] = labelencoder.fit_transform(data[:, 5])
data[:, -1] = labelencoder.fit_transform(data[:, -1].astype(str))
data[:, -2] = labelencoder.fit_transform(data[:, -2].astype(str))

x=pd.DataFrame(data) #input data

y = df.iloc[:,6] # output data
y=y.astype('int')


(375000,)


# Feature selection 

In [286]:
def feature_selection(x,y,k):
    bestfeatures = SelectKBest(score_func=chi2, k=6)
    fit = bestfeatures.fit(x,y)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(x.columns)

    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  #naming the dataframe columns
    print(featureScores.nlargest(5,'Score'))
    d=featureScores.nlargest(5,'Score')
    featuress=np.array(d.iloc[:,0])
    
    return featuress
featurs= feature_selection(x,y,5)

new_data=my_data[:,featurs]



    Specs         Score
10     10  6.515487e+08
0       0  2.281108e+06
3       3  1.822127e+06
2       2  1.318711e+05
4       4  1.266147e+05


array([[0.10824925260089327, 1, 1.3091479683833112, -1.0369310412779704,
        2.206006007014928],
       [-1.2874347675240831, 1, -0.9965794582742259, 1.5513795039507587,
        -0.5156327699695084],
       [-0.5895927574615949, 1, -1.1044911587907094, -1.1663465685394068,
        -0.5214886950437057],
       ...,
       [0.10824925260089327, 1, -1.0942918648633433, -1.1447773139958342,
        -0.5158279674719817],
       [1.5039332727258696, 1, -0.10046843689420125, 1.6376565221250494,
        -0.3586939779810205],
       [0.10824925260089327, 1, -0.7608859302117791, 0.4729167767721216,
        -0.3586939779810205]], dtype=object)

#     split the dataset into training set and testing set  

In [292]:
my_data=np.array(new_data, dtype=object)

y=np.array(y)

# step1: Normalize the new data after feature selection
def normalization(my_data):
    std_dev=np.std(my_data,axis=0,dtype=float)
    mean=np.mean(my_data,axis=0,dtype=float)
   
    zscore =(my_data - mean)/(std_dev+.0001)
        
    return zscore

norm_data=normalization(my_data)

my_data=np.insert(norm_data,0,1,axis=1)

from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(my_data , y, test_size=0.33, random_state=42)


# Model1: Linear Regression

In [296]:
alpha=.00001
eps=.0001

def linear_regression(x_train,y_train,alpha,eps):
    w=np.zeros(x_train.shape[1])
    
    #apply gradient descent algorithm
    
    while(1):
        
        x_t=x_train.transpose()
        b=np.dot(x_t,y_train)
        xtx = np.dot(x_t,x_train)



        trans2=np.dot(xtx,w)

        Djw=np.subtract(trans2,b)    
        intermediate=alpha*Djw
        w_train = np.subtract(w,intermediate)

        
        diff= np.subtract(w_train,w)
        err=np.linalg.norm(diff,2)

        w= np.copy(w_train)
        #calculate error based one w
        if(err<eps):
            break
    return w

def rms_value(x_test,y_test,w_train):
    
    ypred=np.dot(x_test,w_train)
    error=np.subtract(ypred,y_test)
    error=np.sum(error)
    error2 = (error**2)/(x_test.shape[0])
    rms=np.sqrt(error2)
    return(rms)

w=linear_regression(xtrain,ytrain,alpha,eps)
rmse=rms_value(xtest,ytest,w)
print(rmse)

0.05546837087255799


# Model2: Logistic Regression

In [301]:
alpha=.00001
eps=.0001
def logistic_regression(xtrain,ytrain,xtest,ytest,alpha,eps,max_iteration):
    w=np.zeros(xtrain.shape[1])
    w_nxt=np.zeros(xtrain.shape[1])
    j=0
    column=xtrain.shape[1]
    
    def sigmoid(X, T):
        z=np.array(np.dot(X, T),dtype=np.float32)
        return 1.0 / (1.0 + np.exp(-z))
    iteration=0
    # apply gradient ascent algorithm
    while(1):
        
        w_t=w.transpose()
        f=sigmoid(xtrain,w_t)
        diff=np.subtract(ytrain,f)
        for j in range(column):

            mul=np.dot(diff.T,xtrain[:,j])
            w_nxt[j]=w[j]+alpha*(mul)

        w_diff=np.subtract(w,w_nxt)
        err=np.linalg.norm(w_diff)
        print(err)
        w=np.copy(w_nxt)
        iteration=iteration+1
        if(err<eps):
            break
        if(iteration>=max_iteration):
            break
            
    ypred=sigmoidid(xtest,w)
              
    return ypred




def accuracy_score( ytest , ypred ):
    
    tn=tp=fn=fp=0
    for i in range(0,len(yt)):
        if(ypred[i]==0 and ytest[i]==0):
            tn=tn+1
        if(ypred[i]==1 and ytest[i]==1):
            tp=tp+1
        if(ypred[i]==1 and ytest[i]==0):
            fp=fp+1
        if(ypred[i]==0 and ytest[i]==1):
            fn=fn+1

    accuracy=(tn+tp)/(tn+tp+fn+fp)
    sensitivity=tp/(tp+fn)
    specifity=tn/(tn+fp)
    precision=tp/(tp+fp)
    f_score=(2*precision*sensitivity)/(precision+sensitivity)
    
    return(accuracy,sensitivity,specifity,precision,f_score)

ypred=logistic_regression(xtrain,ytrain,xtest,ytest,alpha,eps,100000)
accuracy,sensitivity,specifity,precision,f_score=accuracy_score(ypred,ytest)


0.8062595135749141
0.7581878121039739
0.7244963723953353
0.7046014245702648
0.6937349259031752
0.6879812656565458
0.6849628445450563
0.683377546094413
0.6825396218400301
0.6820926476832815
0.6818516244186767
0.6817201011823059
0.6816474184425422
0.6816067184004163
0.6815836125040622
0.6815703113303994
0.6815625436115412
0.6815579458732236
0.6815551813164052
0.681553497975867
0.6815524594062012
0.6815518082099599
0.6815513970333975
0.6815511310220468
0.6815509606947775
0.6815508479054561
0.6815507739606336
0.681550725030812
0.6815506915361479
0.6815506694663424
0.6815506541651561
0.681550643516551
0.6815506364163247
0.6815506313872248
0.6815506278019521
0.6815506253797193
0.6815506236907619
0.6815506224441921
0.6815506215654452
0.6815506209643996
0.6815506204881947
0.6815506201873995
0.6815506199418067
0.6815506197883201
0.6815506196785965
0.6815506195777764
0.6815506195263097
0.6815506194823967
0.6815506194458244
0.6815506194321189
0.6815506194084991
0.6815506193968388
0.68155061939174

KeyboardInterrupt: 

# Model3: K-Nearest Neighbour

In [263]:

from sklearn.metrics import confusion_matrix 

  
# training a KNN classifier 
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors = 7).fit(xtrain, ytrain) 
  
# accuracy on X_test 
accuracy = knn.score(xtest, ytest) 
print (accuracy )
  
# creating a confusion matrix 
knn_predictions = knn.predict(xtest)  
cm = confusion_matrix(ytest, knn_predictions) 


0.9140202020202021


# Model4: Multiclass Decision Tree Classification

In [264]:

from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 

# training a DescisionTreeClassifier 
from sklearn.tree import DecisionTreeClassifier 

dtree_model = DecisionTreeClassifier(max_depth = 2).fit(xtrain, ytrain) 

dtree_predictions = dtree_model.predict(xtest) 
  
# creating a confusion matrix 
cm = confusion_matrix(ytest, dtree_predictions) 

In [ ]:
cm

# Model5: Decision Tree Regression

In [275]:

from sklearn.tree import DecisionTreeRegressor  
  
# create a regressor object 
regressor = DecisionTreeRegressor(random_state = 0)  
  
# fit the regressor with X and Y data 
regressor.fit(xtrain, ytrain) 

y_pred = regressor.predict(xtest) 
  
# print the predicted price 
print( y_pred) 


[3. 3. 4. ... 3. 4. 4.]
